In [1]:
# Libraries, Variables and Constants
import autogen, os
import json
from dotenv import load_dotenv

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS # requires pip install markdownify

cache_seed = 41 # default seed is 41

print (f"SUPPORTED TEXT_FORMATS: {TEXT_FORMATS}")

SUPPORTED TEXT_FORMATS: ['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']


In [2]:
# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_list.json',
    filter_dict={ "model": {"gpt4-0125preview-128k"} }
)

llm_config={
        "cache_seed": cache_seed,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0  # temperature for sampling        
    }

llm_config

{'cache_seed': 41,
 'config_list': [{'model': 'gpt4-0125preview-128k',
   'api_key': '5948e5b2b4a146cba9940adb3308d731',
   'base_url': 'https://mmopenaiscus.openai.azure.com/',
   'api_type': 'azure',
   'api_version': '2024-02-15-preview'}],
 'temperature': 0}

In [3]:
# import agents
# you need to run pip install "pyautogen[retrievechat]" before running this cell, because 
# retrievechat is an optional feature of the pyautogen package that requires additional dependencies not included in the base package

from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent # requires pypdf

In [4]:
# Create an 'RetrieveAssistantAgent' instance

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config
)

assistant

# Create "ragproxyagent"
By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
`docs_path` is the path to the docs directory. It can also be the path to a single file, or the url to a single file. By default,
it is set to None, which works only if the collection is already created.
`task` indicates the kind of task we're working on. In this example, it's a `code` task.
`chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
`custom_text_types` is a list of file types to be processed. Default is `autogen.retrieve_utils.TEXT_FORMATS`.
This only applies to files under the directories in `docs_path`. Explicitly included files and urls will be chunked regardless of their types.
In this example, we set it to ["mdx"] to only process markdown files. Since no mdx files are included in the `websit/docs`,
no files there will be processed. However, the explicitly included urls will still be processed.

In [5]:
import chromadb
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": [
            "https://raw.githubusercontent.com/microsoft/autogen/main/README.md",
            # "https://drlee.io/harnessing-the-power-of-autogen-and-openai-gpts-for-advanced-code-interpretation-and-development-571ddb6f814c",
            # "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
            # "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
            # os.path.join(os.path.abspath(""), "..", "website", "docs"),
        ],
        "custom_text_types": ["mdx"],
        "chunk_token_size": 2000,
        # "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "mauromi_collection_005",
        "embedding_model": "all-mpnet-base-v2",
        "get_or_create": False,  # set to False if you don't want to reuse an existing collection, but you'll need to remove the collection manually
    },
    code_execution_config=False,  # set to False if you don't want to execute the code
)

ragproxyagent

# Example 1
**Note**: if `sentence_transformers` python package is not installed. Please install it with `pip install sentence_transformers`

In [6]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
code_problem = "What is autogen?"
ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=code_problem,
    search_string="autogen" # used as an extra filter for the embeddings search: in this case, we only want to search documents that contain "autogen"
)  

Trying to create collection.


Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


doc_ids:  [['doc_1', 'doc_0']]
Adding doc_id doc_1 to context.
Adding doc_id doc_0 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is autogen?

Context is: <!-- Each of the [`notebook examples`](https://github.com/microsoft/autogen/tree/main/notebook) may require a specific option to be installed. -->

Even if you are installing and running AutoGen locally outside of docker, the recommendation and default behavior of agents is to perform [code execution](https://microsoft.github.io/autogen/docs/FAQ/#code-execution) in docker. Find more instructions and how to change the default behaviour [here](https://microsoft.github.io/autogen/docs/Installation#code-execution-with-docker-(defau

ChatResult(chat_id=None, chat_history=[{'content': 'You\'re a retrieve augmented chatbot. You answer user\'s questions based on your own knowledge and the\ncontext provided by the user.\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser\'s question is: What is autogen?\n\nContext is: <!-- Each of the [`notebook examples`](https://github.com/microsoft/autogen/tree/main/notebook) may require a specific option to be installed. -->\n\nEven if you are installing and running AutoGen locally outside of docker, the recommendation and default behavior of agents is to perform [code execution](https://microsoft.github.io/autogen/docs/FAQ/#code-execution) in docker. Find more instructions and how to change the default behaviour [here](https://microsoft.github.io/autogen/docs/Installation#code-execution-with-docker-(default)).\n\nFor LLM inference configurations, check the [FAQs](h

AutoGen is a framework developed by Microsoft that enables the creation of next-generation Large Language Model (LLM) applications through a multi-agent conversation framework. It allows for the automation and customization of tasks by facilitating communication between multiple agents, which can include both AI models and human participants. This framework is designed to enhance the capabilities of LLMs by enabling more complex, sophisticated applications and workflows through agent collaboration.

Key features and capabilities of AutoGen include:

- **Multi-Agent Conversations**: AutoGen supports conversations between multiple agents, enabling complex task solving that goes beyond the capabilities of individual LLMs.
- **Customization**: It offers extensive customization options, allowing developers to tailor the behavior and capabilities of agents according to specific application needs.
- **Human Participation**: AutoGen integrates human feedback and input seamlessly into the conversation flow, enhancing the decision-making process and task execution.
- **Enhanced LLM Inferences**: The framework provides enhanced LLM inference features such as caching, error handling, and multi-config inference, aiming to maximize the utility and efficiency of expensive LLMs like ChatGPT and GPT-4.

AutoGen is particularly useful for developers and researchers looking to build advanced LLM applications that require the orchestration of multiple agents and tools, including those that involve human interaction. It supports a wide range of applications across different domains by enabling diverse conversation patterns and complex workflows.

The framework is part of Microsoft's ongoing research and development in AI and LLM technologies, with its capabilities and use cases detailed in several research papers and documentation available on its [official website](https://microsoft.github.io/autogen/).

In [7]:
code_problem = "what are the predefined agents available in the API?"
ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=code_problem,
    search_string="autogen" # used as an extra filter for the embeddings search: in this case, we only want to search documents that contain "autogen"
)  

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


doc_ids:  [['doc_1', 'doc_0']]
Adding doc_id doc_1 to context.
Adding doc_id doc_0 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: what are the predefined agents available in the API?

Context is: <!-- Each of the [`notebook examples`](https://github.com/microsoft/autogen/tree/main/notebook) may require a specific option to be installed. -->

Even if you are installing and running AutoGen locally outside of docker, the recommendation and default behavior of agents is to perform [code execution](https://microsoft.github.io/autogen/docs/FAQ/#code-execution) in docker. Find more instructions and how to change the default behaviour [here](https://microsoft.github.io/autogen/docs/Installati

ChatResult(chat_id=None, chat_history=[{'content': 'You\'re a retrieve augmented chatbot. You answer user\'s questions based on your own knowledge and the\ncontext provided by the user.\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser\'s question is: what are the predefined agents available in the API?\n\nContext is: <!-- Each of the [`notebook examples`](https://github.com/microsoft/autogen/tree/main/notebook) may require a specific option to be installed. -->\n\nEven if you are installing and running AutoGen locally outside of docker, the recommendation and default behavior of agents is to perform [code execution](https://microsoft.github.io/autogen/docs/FAQ/#code-execution) in docker. Find more instructions and how to change the default behaviour [here](https://microsoft.github.io/autogen/docs/Installation#code-execution-with-docker-(default)).\n\nFor LLM inferenc

The predefined agents available in the AutoGen API, as described in the context, include:

1. **AssistantAgent**: This agent is designed to assist in tasks by leveraging Large Language Models (LLMs). It can be customized with specific LLM configurations, including model selection and API keys. The AssistantAgent can perform a variety of tasks, including generating text, answering questions, and more, based on the capabilities of the underlying LLM.

2. **UserProxyAgent**: This agent acts as a proxy for user interactions, allowing for code execution and facilitating the communication between the user and other agents, such as the AssistantAgent. It can be configured to execute code either locally or in a Docker container, depending on the setup preferences.

These agents can communicate with each other to solve complex tasks, and their behavior can be customized through configurations. The framework allows for the integration of additional agents and customization of existing ones to meet specific application needs.

In [8]:
# reset the assistant. This time, we use a simple UserProxyAgent

assistant.reset()
userproxyagent = autogen.UserProxyAgent(
    name              = "userproxyagent", 
    human_input_mode  = "ALWAYS",
    code_execution_config = {"use_docker": False}
)

userproxyagent.initiate_chat(assistant, message="What is autogen?")

userproxyagent (to assistant):

What is autogen?

--------------------------------------------------------------------------------
assistant (to userproxyagent):

"Autogen" can refer to different things depending on the context in which it's used. Here are a few interpretations:

1. **Autogen in Software Development**: In the context of programming and software development, "autogen" typically refers to the process of automatically generating code or documentation. This can be done through tools or scripts that generate boilerplate code, configuration files, or documentation based on a set of inputs or predefined templates. This process is often used to save time, reduce human error, and ensure consistency across large projects.

2. **Autogen in Geographic Information Systems (GIS)**: In the context of GIS and mapping, "autogen" can refer to the automatic generation of geographical features like roads, rivers, and vegetation on maps. This is often used in flight simulation software and

Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': 'What is autogen?', 'role': 'assistant'}, {'content': '"Autogen" can refer to different things depending on the context in which it\'s used. Here are a few interpretations:\n\n1. **Autogen in Software Development**: In the context of programming and software development, "autogen" typically refers to the process of automatically generating code or documentation. This can be done through tools or scripts that generate boilerplate code, configuration files, or documentation based on a set of inputs or predefined templates. This process is often used to save time, reduce human error, and ensure consistency across large projects.\n\n2. **Autogen in Geographic Information Systems (GIS)**: In the context of GIS and mapping, "autogen" can refer to the automatic generation of geographical features like roads, rivers, and vegetation on maps. This is often used in flight simulation software and games to create realistic environments without manu

In [9]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=code_problem, search_string="spark",
)  # search_string is used as an extra filter for the embeddings search, in this case, we only want to search documents that contain "spark".

Number of requested results 20 is greater than number of elements in index 2, updating n_results = 2


doc_ids:  [[]]
No more context, will terminate.
ragproxyagent (to assistant):

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'TERMINATE', 'role': 'assistant'}], summary='', cost=({'total_cost': 0}, {'total_cost': 0}), human_input=[])